<a href="https://colab.research.google.com/github/victorluongo/fiap-techchallenge-fase3/blob/main/Tech_Challenge_03_Llama_3_1_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Google Drive Mount

Montar o Google Drive no ambiente do Google Colab para ler e escrever arquivos, facilitando o armazenamento e a recuperação de datasets e modelos treinados.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import de bibliotecas necessárias para o processamento de dados.

In [2]:
import pandas as pd
import unicodedata
import json
import re

Desativar o logging do `Wandb` para evitar solicitações de API Key e possíveis interrupções no fluxo de execução.

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

###Prepapação do Dataset

In [4]:
def process_and_format_dataset(input_file_path, output_file_path, sample_size=None):

    data = []
    with open(input_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                # Carrega cada linha como um objeto JSON
                item = json.loads(line)
                if 'title' in item and 'content' in item:
                    title = item['title']
                    content = item['content']
                    data.append({'title': title, 'content': content})
            except json.JSONDecodeError:
                # Ignora linhas com erro de decodificação
                pass

    # Converte a lista de dicionários em um DataFrame do pandas
    df = pd.DataFrame(data)

    # Remove registros com valores nulos em 'title' ou 'content'
    df.dropna(subset=['title', 'content'], inplace=True)

    # Remove registros onde 'title' ou 'content' são strings vazias ou contêm apenas espaços em branco
    df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')]

    # Remove duplicatas com base nas colunas 'title' e 'content'
    df.drop_duplicates(subset=['title', 'content'], inplace=True)

    # Reseta o índice do DataFrame
    df.reset_index(drop=True, inplace=True)

    # Função para normalizar o texto
    def normalize_text(text):
        # Remove caracteres de controle
        text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
        # Remove espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Aplica a função de normalização nas colunas 'title' e 'content'
    df['title'] = df['title'].apply(normalize_text)
    df['content'] = df['content'].apply(normalize_text)

    # Cria as colunas 'instruction', 'input' e 'output' para o formato necessário
    df['instruction'] = "Answer the question based on the product title."
    df['input'] = df['title'].apply(lambda x: f"What is '{x}'?")
    df['output'] = df['content']

    # Se 'sample_size' estiver definido, pega uma amostra do DataFrame
    if sample_size is not None:
        df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    # Converte para uma lista de dicionários
    formatted_data = df[['instruction', 'input', 'output']].to_dict(orient='records')

    # Salva o resultado em um arquivo JSON
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4)

    print(f"Dataset processado e salvo em '{output_file_path}'. Total de registros: {len(df)}")

In [26]:
train_dataset = "/content/drive/MyDrive/Fiap/Tech_Challenge_03/trn.json"
train_dataset_output = "/content/drive/MyDrive/Fiap/Tech_Challenge_03/formatted_train_dataset.json"

test_dataset = "/content/drive/MyDrive/Fiap/Tech_Challenge_03/tst.json"
test_dataset_output = "/content/drive/MyDrive/Fiap/Tech_Challenge_03/formatted_test_dataset.json"

Processa e formata os datasets de treinamento e teste com uma amostra de registros

In [27]:
process_and_format_dataset(train_dataset, train_dataset_output, sample_size=250000)  # 250.000 registros para treinamento
process_and_format_dataset(test_dataset, test_dataset_output, sample_size=2500)     # 2.500 registros para teste

Dataset processado e salvo em '/content/drive/MyDrive/Fiap/Tech_Challenge_03/formatted_train_dataset.json'. Total de registros: 250000
Dataset processado e salvo em '/content/drive/MyDrive/Fiap/Tech_Challenge_03/formatted_test_dataset.json'. Total de registros: 2500


###Instalação do Unsloth
Instalação do Unsloth em uma instância **gratuita** do Google Colab, a **Tesla T4**, para o fine-tuning com o modelo opensource **Llama 3.1-8B**.

In [7]:
!pip install "unsloth @ git+https://github.com/unslothai/unsloth.git" unsloth-zoo

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-0ts0jnf2/unsloth_fe2f95c566404de8a92f535a2a7d20a9
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-0ts0jnf2/unsloth_fe2f95c566404de8a92f535a2a7d20a9
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.3 MB/s eta 0:00:00
   ━

###Bibilotecas auxiliares

In [8]:
!pip install peft accelerate bitsandbytes triton --no-deps xformers "trl<0.9.0" dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: trl
    Found existing installation: trl 0.13.0
    Uninstalling trl-0.13.0:
      Successfully uninstalled trl-0.13.0


###Configuração do modelo


Lista de modelos da unsloth quantizados para 4bits para reduzir o consumo de memória e permitir o treinamento do modelo em GPUs com menos VRAM ou gratuitas como a **Tesla T4**.

In [ ]:
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",           # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",         # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",      # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",             # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",                # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",                 # Gemma 2x faster!
]                                                   # More models at https://huggingface.co/unsloth

Definição da quantidade de tokens, tipo de dados, quantização e modelo.

In [9]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

max_seq_length = 128                                  # Define o comprimento máximo das sequências de entrada.
dtype = None                                          # Tipo de dados (também pode ser torch.float16 ou torch.float32).
load_in_4bit = True                                   # Indica que o modelo será quantizado em 4 bits para economizar memória.
model_name = "unsloth/Meta-Llama-3.1-8B"              # Modelo escolhido para executar o fine-tuning

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

###PEFT e LoRA

Configuração do modelo para o treinamento eficiente usando LoRA onde
apenas as camadas lineares do modelo serão treinadas, adaptando o modelo para tarefas específicas mantendo a maioria do conhecimento pré-treinado intacto e reduzindo a quantidade de dados necessário para efetuar o ajuste.


In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                                                     # Rank das matrizes LoRA podendo ser 8, 16, 32, 64, 128. Quanto menor o valor, menos custo computacional e menos poder de ajuste.

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",   # Módulos do modelo que serão adaptados
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,                                            # Controla a escala de atualização das matrizes LoRA
    lora_dropout = 0,                                           # Ajuda a prevenir overfitting em tarefas específicas, onde 0 é otimizado e valores como 0.1 ou 0.2 podem ser úteis para baixo volume de dados.
    bias = "none",                                              # Nenhuma definição necessária para manter a otimização

    use_gradient_checkpointing = "unsloth",                     # Utilização de 30% menos de VRAM e 2x mais rápido para grandes contextos
    random_state = 3407,                                        # Número randômico

    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.12.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


###Prompt

Definição do formato do prompt seguintdo um template consistente para ajudar o modelo a entender melhor a estrutura da tarefa.



In [11]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


Aplicação do template de prompt a cada registro do dataset retornando um dicionário com a chave 'text' contendo os prompts formatados.

In [12]:
EOS_TOKEN = tokenizer.eos_token            # Indicação de fim de linha para evitar repetições
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


##Inferência

Carrega e prepara o modelo para inferência com as otimizações realizadas.

In [13]:
FastLanguageModel.for_inference(model) # Habilita inferência nativa que é 2x mais rápida


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

###Teste de prompt antes do fine-tuning com o dataset formatado


Prompt de teste antes de realizer o fine-tuning do modelo que não encontrará a resposta correta para a pergunta do usuário.

In [ ]:
inputs = tokenizer(
[
    alpaca_prompt.format(
      "Answer the question based on the product title.",  # instruction
      # "What is 'Girls Ballet Tutu Neon Pink'?",           # input
      # "Who was Ayrton Senna?",                            # input
      "Last Pink FLoyd Album?",                           # input
      "",                                                 # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

###Dataset


Mapeamento do dataset formatado e gerado anteriormente pela preparação de dados.

In [17]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Fiap/Tech_Challenge_03/formatted_train_dataset.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

### Treinamento do modelo

Definição de parâmetros do treinamento do modelo utilizando o `SFTTrainer` com `60 steps` para melhorar o desempenho, mas também pode ser utilizado `num_train_epochs=1` para uma execução completa, e desabilitado `max_steps=None`.

In [18]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

output_dir = "/content/drive/MyDrive/Fiap/Tech_Challenge_03/outputs"

args = TrainingArguments(

    per_device_train_batch_size = 1,        # Tamanho do batch por GPU reduzido para 1 para evitar falta de memória
    gradient_accumulation_steps = 2,        # Número de passos de acumulação de gradientes antes de atualizar os pesos do modelo
    warmup_steps = 5,                       # Número de passos iniciais de treinamento nos quais a taxa de aprendizado é gradualmente aumentada.

    # num_train_epochs = 1,                 # Número de épocas de treinamento, desabilitado neste caso, pois o número de passos é limitado pelo max_steps
    max_steps = 60,                         # Número máximo de passos de treinamento a serem executados.
    learning_rate = 2e-4,                   # Taxa de aprendizado inicial do otimizador.

    fp16 = not is_bfloat16_supported(),     # Meio-precisão (16 bits) para econima de memória.
    bf16 = is_bfloat16_supported(),         # Formato bfloat16 (mais estável em algumas arquiteturas de hardware).

    logging_steps = 1,                      # Frequência (em passos) com que as métricas de log são registradas.
    optim = "adamw_8bit",                   # versão de Adam otimizada para economizar memória usando pesos de 8 bits.
    weight_decay = 0.01,                    # Penalidade aplicada aos pesos do modelo para evitar overfitting.
    lr_scheduler_type = "linear",           # Reduz a taxa de aprendizado de forma linear ao longo do treinamento.
    seed = 3407,                            # Controle de aleatoriedade, garantindo a reprodutibilidade.
    output_dir = output_dir,                # Diretório onde os resultados do treinamento (e possivelmente checkpoints) serão armazenados.
    report_to = "none",                     # Relatório e monitoramento de treinamento

    evaluation_strategy="no",               # Sem avaliação durante o treinamento
    save_strategy="no",                     # Não salva checkpoints intermediários

)

trainer = SFTTrainer(

    model = model,                          # Modelo escolhido para executar o fine-tuning
    tokenizer = tokenizer,                  # Tokenizador do modelo que transforma texto em tensores numéricos
    train_dataset = dataset,                # Conjunto de dados de treinamento formatado
    dataset_text_field = "text",            # Nome do campo no dataset que contém o texto que será usado no treinamento
    max_seq_length = max_seq_length,        # Comprimento máximo da sequência de tokens
    dataset_num_proc = 2,                   # Número de processos para carregar e processar os dados do dataset
    packing = False,                        # Pode fazer o treinamento 5x mais rápido com pequenas sequências

    args = args,                            # Argumentos para o treinamento
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/1000000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,000,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 2
\        /    Total batch size = 2 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.052800
2,2.944500
3,3.217500
4,2.760500
5,3.033800
6,2.525700
7,2.012700
8,2.206300
9,1.547100
10,1.393100


TrainOutput(global_step=60, training_loss=1.7219625492890676, metrics={'train_runtime': 102.4688, 'train_samples_per_second': 1.171, 'train_steps_per_second': 0.586, 'total_flos': 660154706239488.0, 'train_loss': 1.7219625492890676, 'epoch': 0.00012})

###Exportação do modelo

Exportação do modelo e o tokenizador treinados para uso futuro sem a necessidade de executar o fine-tuning novamente.

In [19]:
model.save_pretrained("/content/drive/MyDrive/Fiap/Tech_Challenge_03/trained_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Fiap/Tech_Challenge_03/trained_model")

('/content/drive/MyDrive/Fiap/Tech_Challenge_03/trained_model/tokenizer_config.json',
 '/content/drive/MyDrive/Fiap/Tech_Challenge_03/trained_model/special_tokens_map.json',
 '/content/drive/MyDrive/Fiap/Tech_Challenge_03/trained_model/tokenizer.json')

###Utilização do modelo com fine-tuning

Carrega o treinado com o fine-tuning e prepara o modelo para inferência com as otimizações realizadas.

In [20]:
!pip install unsloth
from unsloth import FastLanguageModel

model_name = "/content/drive/MyDrive/Fiap/Tech_Challenge_03/trained_model"

max_seq_length = 128                                  # Define o comprimento máximo das sequências de entrada.
dtype = None                                          # Tipo de dados (também pode ser torch.float16 ou torch.float32).
load_in_4bit = True                                   # Indica que o modelo será quantizado em 4 bits para economizar memória.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    device_map="auto",
)

FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

Prompt depois de realizer o fine-tuning do modelo que encontrará a resposta correta para a pergunta do usuário.

In [25]:
inputs = tokenizer(
[
    alpaca_prompt.format(
      "Answer the question based on the product title.",  # instruction
      # "What is 'Girls Ballet Tutu Neon Pink'?",           # input
      "Who was Ayrton Senna?",                            # input
      # "Last Pink FLoyd Album?",                           # input

      "",                                                 # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=250)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the product title.

### Input:
Who was Ayrton Senna?

### Response:
Ayrton Senna da Silva (21 March 1960 - 1 May 1994) was a Brazilian racing driver and three-time Formula One World Champion. He is the third most successful F1 driver of all time in terms of race wins, behind Michael Schumacher and Alain Prost. Senna is considered by many to be one of the greatest drivers of all time. He won the title in 1988, 1990 and 1991 in a McLaren-Honda, and is the only driver to win the title in a car manufactured by a company based in Brazil. Senna's death in a car accident at the 1994 San Marino Grand Prix led to a change of the rules to improve safety. In 1999, he was posthumously elected to the FIA Hall of Fame.
